# GPU Setup
Following code checks if a GPU is available for use with PyTorch and sets the device accordingly. If a GPU is available, it assigns the device to "cuda" and prints the GPU's name. If not, it defaults to using the CPU.

In [1]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: NVIDIA GeForce RTX 4070 Ti SUPER


# Libraries
Following code imports necessary libraries for interacting with mongodb and sentencetransformers library

In [2]:

import pymongo
import tensorflow as tf
from sentence_transformers import SentenceTransformer

c:\Users\Admin\anaconda3\envs\pytorch_env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# MongoDB Connection
Following code connects to the mongodb database and gets access to the collection

In [3]:
# MongoDB Connection String
client = pymongo.MongoClient("mongodb+srv://<user>:<password>@cluster0.yzkq3xh.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0")
db = client.Thesis
collection = db.Documents

In [5]:
# Query documents where 'embed' field is either missing or set to null
missing_embeds = collection.find({'$or': [{'embed': {'$exists': False}}, {'embed': None}]})

# Initialize a counter
missing_count = 0

# Iterate over the documents, print details, and count them
for doc in missing_embeds:
    print(f"Document ID: {doc['_id']}, Title/Identifier: {doc.get('identifier', doc.get('title'))}")
    missing_count += 1

# Print the total count of documents missing the embed field or having it set to null
print(f"Total number of documents missing 'embed': {missing_count}")


Document ID: 667e2316530f8ab3a5ba15ee, Title/Identifier: G.R. No. 203754
Document ID: 667e2316530f8ab3a5ba15e5, Title/Identifier: A.M. No. RTJ-16-2462 [Formerly OCA IPI No. 14-4311-RTJ]
Document ID: 667e2316530f8ab3a5ba15f8, Title/Identifier: G.R. No. 214546
Document ID: 667e2316530f8ab3a5ba15f2, Title/Identifier: G.R. No. 210503
Document ID: 667e2316530f8ab3a5ba15e4, Title/Identifier: A.M. No. RTJ-10-2250 (Formerly A.M. No. 08-08-460-RTC)
Document ID: 667e2316530f8ab3a5ba15e9, Title/Identifier: G.R. No. 198404
Document ID: 667e2316530f8ab3a5ba15ea, Title/Identifier: G.R. No. 198867
Document ID: 667e2316530f8ab3a5ba15ec, Title/Identifier: G.R. No. 199666
Document ID: 667e2316530f8ab3a5ba15f0, Title/Identifier: G.R. No. 205752
Document ID: 667e2316530f8ab3a5ba15f4, Title/Identifier: G.R. No. 211422
Document ID: 667e2316530f8ab3a5ba15f5, Title/Identifier: G.R. No. 211559
Document ID: 667e2316530f8ab3a5ba15f6, Title/Identifier: G.R. No. 212215
Document ID: 667e2316530f8ab3a5ba15ef, Title/

# Model
Following code uses main model for embedding/retrieval task

In [6]:
# Ensure GPU usage for SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", device=device, trust_remote_code=True)

# Embedding Function
Following code is the function called when converting text to embeddings

In [7]:
def generate_embedding(text):
  return model.encode(text).tolist()

## Embedding Documents

 Following code uses an embedding model to convert the text field into vector representations and inserts them to the 'embed' field

In [8]:
#concat the list of text from db
skip_count = 0
counter = skip_count

try:
    for doc in collection.find({'text': {"$exists": True}, '$or': [{'embed': {'$exists': False}}, {'embed': None}]}):
        # Increment counter
        counter += 1

        # Concatenate the list of strings into a single text
        text_list = doc['text']
        concatenated_text = ' '.join(text_list)  # Join list of strings into one string

        # Embed the concatenated text
        embedding = generate_embedding(concatenated_text)

        # Add the embedding to a new field 'embed'
        collection.update_one(
            {'_id': doc['_id']},  # Select the document by its unique ID
            {'$set': {'embed': embedding}}  # Add the new 'embed' field with the embedding
        )

        # Print the successful embedding
        print(f"Successfully embedded data from {str(doc.get('identifier', doc.get('title')))} = counter: {counter}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    print(f"Number of documents processed: {counter}")


C:\Users\Admin\.cache\huggingface\modules\transformers_modules\Alibaba-NLP\new-impl\40ced75c3017eb27626c9d4ea981bde21a2662f4\modeling.py:579: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Successfully embedded data from G.R. No. 203754 = counter: 40848
Successfully embedded data from A.M. No. RTJ-16-2462 [Formerly OCA IPI No. 14-4311-RTJ] = counter: 40849
Successfully embedded data from G.R. No. 214546 = counter: 40850
Successfully embedded data from G.R. No. 210503 = counter: 40851
Successfully embedded data from A.M. No. RTJ-10-2250 (Formerly A.M. No. 08-08-460-RTC) = counter: 40852
Successfully embedded data from G.R. No. 198404 = counter: 40853
Successfully embedded data from G.R. No. 198867 = counter: 40854
Successfully embedded data from G.R. No. 199666 = counter: 40855
Successfully embedded data from G.R. No. 205752 = counter: 40856
Successfully embedded data from G.R. No. 211422 = counter: 40857
Successfully embedded data from G.R. No. 211559 = counter: 40858
Successfully embedded data from G.R. No. 212215 = counter: 40859
Successfully embedded data from G.R. No. 204232 = counter: 40860
Successfully embedded data from G.R. No. 221709 = counter: 40861
Successfull

## Summarizing Documents
Following code uses BART pipeline to create summaries for the text, and summarizes them by segments to make a comprehensive summary

In [4]:
from transformers import pipeline
from transformers import AutoTokenizer
summarization_pipe_gpu = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=0)
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

c:\Users\Admin\anaconda3\envs\pytorch_env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
from datasets import Dataset

def validate_input(text):
    """Ensure that the input text is valid for summarization."""
    if not isinstance(text, str):
        raise ValueError("Input text must be a string.")
    if len(text.strip()) == 0:
        raise ValueError("Input text is empty or only contains whitespace.")
    return text

def chunk_text_by_tokens(text, max_tokens=512):
    """Chunk text into parts of at most max_tokens tokens."""
    tokens = tokenizer(text, return_tensors="pt", truncation=False, add_special_tokens=False)['input_ids'][0]
    
    # Ensure no chunk exceeds max_tokens length
    token_chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    
    # Decode the token chunks back to text
    text_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in token_chunks]
    
    return text_chunks

def process_texts(text_chunks):
    """Process each chunk individually and generate summaries with dynamic max_length."""
    summaries = []
    try:
        for chunk in text_chunks:
            input_length = len(tokenizer(chunk)['input_ids'])
            # Set max_length dynamically based on input length
            max_length = int(input_length * 0.5) if input_length > 1 else 1  # Ensure max_length is at least 1
            summary = summarization_pipe_gpu(chunk, max_length=max_length)
            summaries.append(summary[0]['summary_text'])
    except Exception as e:
        print(f"Error during processing: {e}")
    
    # Concatenate all summaries into a single summary
    return ' '.join(summaries)

def generate_summary(text):
    """Generate a summary using the GPU for each chunk."""
    text = validate_input(text)  # Validate the input text
    text_chunks = chunk_text_by_tokens(text, max_tokens=512)  # Chunk the text into smaller 512-token parts
    
    # Process each chunk and generate a summary
    return process_texts(text_chunks)

# Process documents to generate summaries
# Process documents to generate summaries
counter = 6722

# Start an explicit session to avoid session timeout issues
try:
    with client.start_session() as session:
        cursor = collection.find(
            {'text': {"$exists": True}, '$or': [{'summary': {'$exists': False}}, {'summary': None}, {'summary': ""}]}, 
            no_cursor_timeout=True, 
            session=session
        )

        # Use a 'with' block to ensure the cursor is properly closed after use
        with cursor as docs:
            for doc in docs:
                counter += 1

                # Ensure that `text` is either a string or a list of strings
                text_field = doc['text']
                if isinstance(text_field, list):
                    # Concatenate the list of strings into a single text
                    concatenated_text = ' '.join(text_field)
                elif isinstance(text_field, str):
                    # If it's a pure string, use it as is
                    concatenated_text = text_field
                else:
                    raise ValueError(f"Expected 'text' field to be a list of strings or a string, but got {type(text_field)}.")

                # Generate a summary for the concatenated text, using GPU
                summary = generate_summary(concatenated_text)

                # Update the document with the generated summary
                collection.update_one(
                    {'_id': doc['_id']},  # Select the document by its unique ID
                    {'$set': {'summary': summary}},  # Add the new 'summary' field
                    session=session  # Use the explicit session here
                )

                print(f"Successfully summarized data from {str(doc.get('identifier', doc.get('title')))} = counter: {counter}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    print(f"Number of documents processed: {counter}")


Your min_length=56 must be inferior than your max_length=33.


Successfully summarized data from G.R. No. 119745 = counter: 6723
Successfully summarized data from G.R. No. 123673 = counter: 6724
Successfully summarized data from G.R. No. 125221 = counter: 6725


Your min_length=56 must be inferior than your max_length=2.


Successfully summarized data from G.R. No. 118041 = counter: 6726


Your min_length=56 must be inferior than your max_length=48.


...

Your min_length=56 must be inferior than your max_length=42.


Successfully summarized data from G.R. No. 47013 = counter: 9591
Successfully summarized data from G.R. Nos. 137017-18 = counter: 9592
Successfully summarized data from G.R. Nos. 124680-81 = counter: 9593
Successfully summarized data from A.M. No. MTJ-00-1245 = counter: 9594
Successfully summarized data from A.M. No. P-00-1360 = counter: 9595


Your min_length=56 must be inferior than your max_length=26.


Successfully summarized data from A.M. No. RTJ-00-1522 = counter: 9596
Successfully summarized data from G. R. No. 116332 = counter: 9597
Successfully summarized data from G.R. No. 100718-19 = counter: 9598
Successfully summarized data from G.R. Nos. 131384-87 = counter: 9599
Successfully summarized data from G. R. No. 133132 = counter: 9600
Successfully summarized data from G.R. No. 102706 = counter: 9601
Successfully summarized data from G.R. No. 109376 = counter: 9602
Successfully summarized data from G.R. No. 112139 = counter: 9603
Successfully summarized data from G.R. No. 113684 = counter: 9604
Successfully summarized data from G.R. No. 119652 = counter: 9605
Number of documents processed: 9606


KeyboardInterrupt: 

In [14]:
from datasets import Dataset

def validate_input(text):
    """Ensure that the input text is valid for summarization."""
    if not isinstance(text, str):
        raise ValueError("Input text must be a string.")
    if len(text.strip()) == 0:
        raise ValueError("Input text is empty or only contains whitespace.")
    return text

def chunk_text_by_tokens(text, max_tokens=512):
    """Chunk text into parts of at most max_tokens tokens."""
    tokens = tokenizer(text, return_tensors="pt", truncation=False, add_special_tokens=False)['input_ids'][0]
    
    # Ensure no chunk exceeds max_tokens length
    token_chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    
    # Decode the token chunks back to text
    text_chunks = [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in token_chunks]
    
    return text_chunks

def summarize_batch(batch_chunks):
    """Summarize a batch of chunks."""
    # Set max_length dynamically based on the chunk lengths in the batch
    input_lengths = [len(tokenizer(chunk)['input_ids']) for chunk in batch_chunks]
    max_lengths = [int(length * 0.5) if length > 1 else 1 for length in input_lengths]
    
    # Generate summaries in a batch
    summaries = summarization_pipe_gpu(batch_chunks, max_length=max(max_lengths), batch_size=len(batch_chunks))
    
    # Extract the summary text from the results
    return [summary['summary_text'] for summary in summaries]

def process_texts_in_batches(text_chunks, batch_size=8):
    """Process the text chunks in batches."""
    summaries = []
    
    # Iterate over text chunks in batches
    for i in range(0, len(text_chunks), batch_size):
        batch_chunks = text_chunks[i:i + batch_size]  # Get a batch of chunks
        try:
            # Summarize the batch
            batch_summaries = summarize_batch(batch_chunks)
            summaries.extend(batch_summaries)
        except Exception as e:
            print(f"Error during batch processing: {e}")
    
    # Concatenate all summaries into a single summary
    return ' '.join(summaries)

def generate_summary(text):
    """Generate a summary using the GPU for each chunk in batches."""
    text = validate_input(text)  # Validate the input text
    text_chunks = chunk_text_by_tokens(text, max_tokens=512)  # Chunk the text into smaller 512-token parts
    
    # Process the chunks in batches and generate a summary
    return process_texts_in_batches(text_chunks, batch_size=8)  # Adjust batch_size based on GPU memory

# Process documents to generate summaries
counter = 26009

# Start an explicit session to avoid session timeout issues
try:
    with client.start_session() as session:
        cursor = collection.find(
            {'text': {"$exists": True}, '$or': [{'summary': {'$exists': False}}, {'summary': None}, {'summary': ""}]}, 
            no_cursor_timeout=True, 
            session=session
        )

        # Use a 'with' block to ensure the cursor is properly closed after use
        with cursor as docs:
            for doc in docs:
                counter += 1

                # Ensure that `text` is either a string or a list of strings
                text_field = doc['text']
                if isinstance(text_field, list):
                    # Concatenate the list of strings into a single text
                    concatenated_text = ' '.join(text_field)
                elif isinstance(text_field, str):
                    # If it's a pure string, use it as is
                    concatenated_text = text_field
                else:
                    raise ValueError(f"Expected 'text' field to be a list of strings or a string, but got {type(text_field)}.")

                # Generate a summary for the concatenated text, using GPU and batch processing
                summary = generate_summary(concatenated_text)

                # Update the document with the generated summary
                collection.update_one(
                    {'_id': doc['_id']},  # Select the document by its unique ID
                    {'$set': {'summary': summary}},  # Add the new 'summary' field
                    session=session  # Use the explicit session here
                )

                print(f"Successfully summarized data from {str(doc.get('identifier', doc.get('title')))} = counter: {counter}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    print(f"Number of documents processed: {counter}")


Successfully summarized data from G.R. No. 203969 = counter: 26010
Successfully summarized data from G.R. No. 211638 = counter: 26011
Successfully summarized data from G.R. No. 208015 = counter: 26012
Successfully summarized data from G.R. No. 208802 = counter: 26013
Successfully summarized data from G.R. No. 212861 = counter: 26014
Successfully summarized data from G.R. No. 213014 = counter: 26015
Successfully summarized data from G.R. Nos. 153745-46 = counter: 26016
Successfully summarized data from G.R. No. 206910 = counter: 26017
Successfully summarized data from G.R. No. 214057 = counter: 26018
Successfully summarized data from G.R. No. 210841 = counter: 26019
Successfully summarized data from G.R. No. 214506 = counter: 26020
Successfully summarized data from G.R. No. 215313 = counter: 26021
Successfully summarized data from G.R. Nos. 204481-82 = counter: 26022
Successfully summarized data from G.R. No. 211145 = counter: 26023
Successfully summarized data from G.R. No. 215319 = co

Your min_length=56 must be inferior than your max_length=46.


Successfully summarized data from G.R. No. 184397 = counter: 26048
Successfully summarized data from G.R. No. 188794 = counter: 26049
Successfully summarized data from G.R. No. 187727 = counter: 26050
Successfully summarized data from G.R. No. 191432 = counter: 26051
Successfully summarized data from G.R. No. 186522 = counter: 26052
Successfully summarized data from G.R. No. 182534 = counter: 26053
Successfully summarized data from G.R. No. 182151 = counter: 26054
Successfully summarized data from G.R. No. 187418 = counter: 26055
Successfully summarized data from G.R. No. 191479 = counter: 26056
Successfully summarized data from G.R. No. 186204 = counter: 26057
Successfully summarized data from G.R. No. 188639 = counter: 26058
Successfully summarized data from G.R. No. 198057 = counter: 26059
Successfully summarized data from G.R. No. 191641 = counter: 26060
Successfully summarized data from G.R. No. 198426 = counter: 26061
Successfully summarized data from G.R. No. 199617 = counter: 2

Your min_length=56 must be inferior than your max_length=26.


Successfully summarized data from G.R. No. 209835 = counter: 26092
Successfully summarized data from G.R. No. 209587 = counter: 26093
Successfully summarized data from G.R. No. 205505 = counter: 26094
Successfully summarized data from G.R. No. 207964 = counter: 26095
Successfully summarized data from G.R. No. 211160 = counter: 26096
Successfully summarized data from G.R. No. 212764 = counter: 26097
Successfully summarized data from G.R. No. 212940 = counter: 26098
Successfully summarized data from A.C. No. 11128 = counter: 26099
Successfully summarized data from A.C. No. 6934 = counter: 26100
Successfully summarized data from A.C. No. 8172 = counter: 26101
Successfully summarized data from A.M. No. 12-8-59-MCTC = counter: 26102
Successfully summarized data from A.M. No. RTJ-16-2455 (Formerly OCA I.P.I. No.10-3443-RTJ) = counter: 26103
Successfully summarized data from G.R. No. 167838 = counter: 26104
Successfully summarized data from G.R. No. 207161 = counter: 26105
Successfully summar

...

Successfully summarized data from G.R. No. 220903 = counter: 33274
Successfully summarized data from G.R. No. 215494 = counter: 33275
Successfully summarized data from G.R. No. 222957 = counter: 33276
Successfully summarized data from G.R. No. 206863 = counter: 33277
Successfully summarized data from G.R. No. 212581 = counter: 33278


Your min_length=56 must be inferior than your max_length=3.


Successfully summarized data from G.R. No. 228232 = counter: 33279
Successfully summarized data from G.R. No. 228127 = counter: 33280
Successfully summarized data from G.R. No. 229706 = counter: 33281
Successfully summarized data from G.R. No. 230704 = counter: 33282
Successfully summarized data from G.R. No. 230013 = counter: 33283
Successfully summarized data from G.R. No. 224550 = counter: 33284
Successfully summarized data from G.R. No. 231546 = counter: 33285


Your min_length=56 must be inferior than your max_length=21.


Successfully summarized data from G.R. No. 236525 = counter: 33286
Successfully summarized data from G.R. No. 235099 = counter: 33287
Successfully summarized data from G.R. No. 238877 = counter: 33288
Successfully summarized data from G.R. No. 236924 = counter: 33289
Successfully summarized data from G.R. No. 243133 = counter: 33290
Successfully summarized data from G.R. No. 246933 = counter: 33291
Successfully summarized data from G.R. No. 249684 = counter: 33292
Successfully summarized data from G.R. No. 249434 = counter: 33293
Successfully summarized data from G.R. No. 247339 = counter: 33294
Successfully summarized data from G.R. No. 247646 = counter: 33295
Successfully summarized data from G.R. No. 244681 = counter: 33296
Successfully summarized data from G.R. No. 248650 = counter: 33297
Successfully summarized data from G.R. No. 249387 = counter: 33298
Successfully summarized data from G.R. No. 250219 = counter: 33299
Successfully summarized data from G.R. No. 255085 = counter: 3